In [1]:
import pandas as pd
import numpy as np

In [2]:
#Read Yelp csv file
path='D:/Adarsh_Laptop/studetmater/python/pycon-2016-tutorial/data/yelp.csv'
data = pd.read_csv(path)

In [3]:
#select only rating 1 and 5 from stars columns of yelp.csv data
data = data[(data.stars == 5) | (data.stars == 1)] 
#Retreive only text and stars columns
X_data = data['text']
y_data = data['stars']
#checking for missing data and found none
X_data.isnull().sum()
y_data.isnull().sum()

0

In [4]:
X_data.shape

(4086L,)

In [5]:
y_data.shape

(4086L,)

In [6]:
#Split the data into train and test set

from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X_data,y_data,test_size =0.3,random_state=42)
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(2860L,)
(1226L,)
(2860L,)
(1226L,)


In [17]:
#CountVectorizer shall be used to work on text data
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

<2860x16218 sparse matrix of type '<type 'numpy.int64'>'
	with 220980 stored elements in Compressed Sparse Row format>

In [18]:
#Vectorize Test data
X_test_dtm = vect.transform(X_test)

In [19]:
#Encode 1 and 5 rating
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y_train=lb.fit_transform(y_train)
y_test=lb.fit_transform(y_test)

In [20]:
#using GridsearchCV for parameter selection and tuning.

from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV


grid = GridSearchCV(LogisticRegression(),param_grid={'C':np.arange(1,20,0.2),'penalty':['l1','l2']},cv=10)
grid.fit(X_train_dtm,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': array([  1. ,   1.2, ...,  19.6,  19.8])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [21]:
# predict 
grid.predict(X_test_dtm)

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [22]:
# Best params
grid.best_params_

{'C': 4.1999999999999993, 'penalty': 'l2'}

In [23]:
#Best Score
grid.best_score_

0.93356643356643354

In [24]:
#Lest use C = 4.19 and penalty l2
log_reg = LogisticRegression(C=4.19,penalty='l2')
log_reg.fit(X_train_dtm,y_train)

LogisticRegression(C=4.19, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [25]:
#predict
logr_pred = log_reg.predict(X_test_dtm)
print logr_pred

[0 1 1 ..., 1 1 1]


In [26]:
#lets analyse 25 elements of predicted and test y labels , 0 means 1 rating and 1 means 5 rating 
print'Predicted',(logr_pred[:25])
print'True     ',(y_test[:25])

Predicted [0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
True      [0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1]


In [27]:
#Lets calculate the accuracy 
from sklearn import metrics
print 'accuracy score',metrics.accuracy_score(y_test,logr_pred)

accuracy score 0.927406199021


In [28]:
#Lets see the confusion metrix 
conf_matrix=metrics.confusion_matrix(y_test,logr_pred)
print 'confusion matrix \n',conf_matrix

confusion matrix 
[[173  60]
 [ 29 964]]


In [29]:
#Lets visualize 
pd.crosstab(y_test, logr_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,173,60,233
1,29,964,993
All,202,1024,1226


In [30]:
#Lets see AUC 
auc_val = metrics.roc_auc_score(y_test,logr_pred)
print 'ROC_AUC_SCORE:- ', auc_val

ROC_AUC_SCORE:-  0.856642419685


In [31]:
#Sensitivity : is the classifier to detecting positive instances , also True Positive Rate" or "Recall"
recall_logr= metrics.recall_score(y_test,logr_pred)
print 'Recall score',recall_logr

Recall score 0.970795568983


In [32]:
#Precision: When a positive value is predicted, how often is the prediction correct?
#How "precise" is the classifier when predicting positive instances?

Precision_logr= metrics.precision_score(y_test,logr_pred)
print 'Precision Score', Precision_logr

Precision Score 0.94140625


In [33]:
#print rating 1 reviews which are falsely callisified as positive i.e rating 5 ,diplay False Postive review comments
X_test[(logr_pred == 1) & (y_test == 0)]

389     I really don't get why everyone is raving this...
2060                This place is closed.  Good riddance.
715     What's with the cheese?  It isn't even Velveta...
2084    Easily the worst "burrito" I have ever had.\nI...
5977    You want good food? You'd be better off smuggl...
9441    It was my Birthday and my wife, her mom, a dea...
6222    My mother always told me, if I didn't have any...
2297    This is a sad excuse for a gay resort.  Sort o...
487     Would have to agree with Rana A below... I thi...
79      This is my first year participating in Arizona...
8507                         This location is now closed.
8569    This place has the oiliest food I've ever eate...
3677    Hands down, the single worst public mass trans...
7035    Totally excited to try this place out, my gran...
8626                       sslowest drive through ever!!!
4686                UPDATE: This location is closed. Boo!
8384         in addition to sucking, it's also expensive.
3482    Door c

In [34]:
#Lets inspect 389 review 
X_test[389]

"I really don't get why everyone is raving this place.  I've been here twice and ordered the carne asada burrito.  The portions are really small for the price that they charge.  There is too much tortilla for the meat.  I think this place is overrated and that there are better places for carne asada in the valley."

In [35]:
#print rating 5 reviews which are falsely callisified as negative i.e rating 1,diplay True Negative review comments
X_test[(logr_pred == 0) & (y_test == 1)]

2504    I've passed by prestige nails in walmart 100s ...
2958    I stopped in with a friend on a Saturday night...
3790    My mother and I were actually getting out of t...
9636    OK OK... as a Proud Italian I hope my momma do...
3075                       Unfortunately Out of Business.
8648    This 5 star rating is mostly for their boba! T...
5699    Staying at the Royal Palms has been on my buck...
8434    a weekend of debauchery was spent here when i ...
763     Here's the deal. I said I was done with OT, bu...
1404    Excellent customer service, super clean, and t...
3855    Last week, I had my first appointment at Pure ...
180     Hippies scare me.     So in my preparation to ...
7240    Love this gym. Not many know it's here..lunch ...
5250    We checked out the prices at The Dump, the sal...
1857    I stopped by a Friday morning a few months pas...
2902    Southwest blows its competitors so far out of ...
5374    Last time I went to Zinburger I was wearing a ...
3987    I'm ve

In [36]:
#Lets inspect 2504 review 
X_test[2504]

"I've passed by prestige nails in walmart 100s of times but never really thought of having a pedicure there (even though they are always busy!) As I stared at my feet, long overdue for a pedicure, I thought it was about time to try them...since walmart rarely let's me down why should the nail salon inside?\n\nTo my surprise I got a wonderful pedicure or $23 not too bad this day in age...my to mention it was just as good as going to the more upscale salon just across the street! \n\nI'm glad to be the first to review them they deserve it! Now if only they did facials at walmart and hair I'd be set!"

In [38]:
#Lets print classifcation report
class_report = metrics.classification_report(y_test,logr_pred)
print class_report

             precision    recall  f1-score   support

          0       0.86      0.74      0.80       233
          1       0.94      0.97      0.96       993

avg / total       0.93      0.93      0.93      1226



In [ ]:
#Upto this point Logistic regression seems to be doing good than Naive Bayes 